# Quick Start

See the API for more detailed information, examples, formulas, and references for each function.

In [2]:
import numpy as np
import xarray as xr
import xskillscore as xs
np.random.seed(seed=42)

Here, we generate some sample gridded data. Our data has three time steps, and a 4x5 latitude/longitude grid. `obs` replicates some verification data and `fct` some forecast (e.g. from a statistical or dynamical model).

In [24]:
obs = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.arange(4),
           np.arange(5),
       ],
       dims=["time", "lat", "lon"],
       name='var'
   )
fct = obs.copy()
fct.values = np.random.rand(3, 4, 5)

## Deterministic Metrics

`xskillscore` offers a suite of correlation-based and distance-based deterministic metrics.

### Correlation-Based 

* Pearson Correlation (`pearson_r`)
* Pearson Correlation p value (`pearson_r_p_value`)
* Pearson Correlation effective p value (`pearson_r_eff_p_value`)
* Spearman Correlation (`spearman_r`)
* Spearman Correlation p value (`spearman_r_p_value`)
* Spearman Correlation effective p value (`spearman_r_eff_p_value`)
* Effective Sample Size (`effective_sample_size`)
* Coefficient of Determination (`r2`)

### Distance-Based

* Root Mean Squared Error (`rmse`)
* Mean Squared Error (`mse`)
* Mean Absolute Error (`mae`)
* Median Absolute Error (`median_absolute_error`)
* Symmetric Mean Absolute Percentage Error (`smape`)
* Mean Absolute Percentage Error (`mape`)

Calling the functions is very straight-forward. All deterministic functions take the form `func(a, b, dim=None, **kwargs)`. **Notice that the original dataset is reduced by the dimension passed.** I.e., since we passed `time` as the dimension here, we are returned an object with dimensions `(lat, lon)`. For correlation metrics `dim` cannot be `[]`.

In [25]:
r = xs.pearson_r(obs, fct, dim='time')
print(r)

<xarray.DataArray 'var' (lat: 4, lon: 5)>
array([[ 0.94713667,  0.41375151,  0.86022013,  0.57183562, -1.        ],
       [ 0.62422846,  0.99816027,  0.05422668,  0.61581821,  0.88283853],
       [ 0.7983406 , -0.83728729, -0.2522497 ,  0.66487784, -0.96751933],
       [ 0.75878658,  0.97991854, -0.15064652,  0.1626912 ,  0.68374901]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [26]:
p = xs.pearson_r_p_value(obs, fct, dim="time")
print(p)

<xarray.DataArray 'var' (lat: 4, lon: 5)>
array([[2.07923884e-01, 7.28436473e-01, 3.40652131e-01, 6.12463018e-01,
        4.68700103e-05],
       [5.70826877e-01, 3.86223113e-02, 9.65461280e-01, 5.77651145e-01,
        3.11258970e-01],
       [4.11422976e-01, 3.68280414e-01, 8.37659135e-01, 5.36967178e-01,
        1.62700993e-01],
       [4.51585088e-01, 1.27797462e-01, 9.03728940e-01, 8.95965134e-01,
        5.20696629e-01]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also specify multiple axes for deterministic metrics. Here, we apply it over the latitude and longitude dimension (a pattern correlation).

In [27]:
r = xs.pearson_r(obs, fct, dim=["lat", "lon"])
print(r)

<xarray.DataArray 'var' (time: 3)>
array([0.27220668, 0.16956987, 0.10558748])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


All deterministic metrics except for `pearson_r_eff_p_value`, `spearman_r_eff_p_value`, and `effective_sample_size` can take the kwarg `weights=...`. `weights` should be a DataArray of the size of the reduced dimension (e.g., if time is being reduced it should be of length 3 in our example).

Weighting is a common practice when working with observations and model simulations of the Earth system. When working with rectilinear grids, one can weight the data by the cosine of the latitude, which is maximum at the equator and minimum at the poles (as in the below example). More complicated model grids tend to be accompanied by a cell area varaible, which could also be passed into this function.

In [28]:
obs2 = xr.DataArray(
       np.random.rand(3, 180, 360),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.linspace(-89.5, 89.5, 180),
           np.linspace(-179.5, 179.5, 360),
       ],
       dims=["time", "lat", "lon"],
    )
fct2 = obs2.copy()
fct2.values = np.random.rand(3, 180, 360)

In [29]:
# make weights as cosine of the latitude and broadcast
weights = np.cos(np.deg2rad(obs2.lat))
_, weights = xr.broadcast(obs2, weights)

# Remove the time dimension from weights
weights = weights.isel(time=0)

In [30]:
r_weighted = xs.pearson_r(obs2, fct2, dim=["lat", "lon"], weights=weights)
print(r_weighted)

<xarray.DataArray (time: 3)>
array([ 0.00043186,  0.0009788 , -0.00169057])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [31]:
r_unweighted = xs.pearson_r(obs2, fct2, dim=["lat", "lon"], weights=None)
print(r_unweighted)

<xarray.DataArray (time: 3)>
array([ 0.00110362, -0.00065519, -0.00255675])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


You can also pass the optional boolean kwarg `skipna=...`. If `True`, ignore any NaNs (pairwise) in `a` and `b` when computing the result. If `False`, return NaNs anywhere there are pairwise NaNs.

In [32]:
obs_with_nans = obs.where(obs.lat > 1)
fct_with_nans = fct.where(fct.lat > 1)
print(obs_with_nans)

<xarray.DataArray 'var' (time: 3, lat: 4, lon: 5)>
array([[[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.83780332, 0.64109487, 0.09950929, 0.81172325, 0.27050025],
        [0.45011662, 0.14055246, 0.95677322, 0.46467022, 0.00945499]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.02364714, 0.283504  , 0.55472714, 0.62589189, 0.75234069],
        [0.55285949, 0.64947159, 0.56332061, 0.48680828, 0.87845533]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.90068156, 0.72526403, 0.49445481, 0.44512377, 0.91854395],
        [0.89473176, 0.30008359, 0.54757183, 0.54372023, 0.22299203]]])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00
  * lat      (lat) 

In [33]:
mae_with_skipna = xs.mae(obs_with_nans, fct_with_nans, dim=['lat', 'lon'], skipna=True)
print(mae_with_skipna)

<xarray.DataArray 'var' (time: 3)>
array([0.38820409, 0.25811598, 0.27111085])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [34]:
mae_without_skipna = xs.mae(obs_with_nans, fct_with_nans, dim=['lat', 'lon'], skipna=False)
print(mae_without_skipna)

<xarray.DataArray 'var' (time: 3)>
array([nan, nan, nan])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Probabilistic Metrics

`xskillscore` offers a suite of probabilistic metrics, mostly ported from `properscoring`.

* Ranked Probability Score (`rps`)
* Continuous Ranked Probability Score with the ensemble distribution (`crps_ensemble`)
* Continuous Ranked Probability Score with a Gaussian distribution (`crps_gaussian`)
* Continuous Ranked Probability Score with numerical integration of the normal distribution (`crps_quadrature`)
* Brier scores of an ensemble for exceeding given thresholds (`threshold_brier_score`)
* Brier Score (`brier_score`)
* Discrimination (`discrimination`)


We now create some data with an ensemble member dimension. In this case, we envision an ensemble forecast with multiple members to validate against our theoretical observations.

In [35]:
obs3 = xr.DataArray(
       np.random.rand(4, 5),
       coords=[np.arange(4), np.arange(5)],
       dims=["lat", "lon"],
       name='var'
   )
fct3 = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[np.arange(3), np.arange(4), np.arange(5)],
       dims=["member", "lat", "lon"],
       name='var'
   )

Continuous Ranked Probability Score with the ensemble distribution. Pass `dim=[]` to get the same behaviour as `properscoring.crps_ensemble` without any averaging over `dim`.

In [36]:
crps_ensemble = xs.crps_ensemble(obs3, fct3, dim=[])
print(crps_ensemble)

<xarray.DataArray 'var' (lat: 4, lon: 5)>
array([[0.13282811, 0.40636887, 0.06729522, 0.44657515, 0.65109678],
       [0.04522958, 0.19518948, 0.36626763, 0.17518107, 0.06002814],
       [0.22024925, 0.24672944, 0.54901602, 0.01377921, 0.24542364],
       [0.1227431 , 0.69763868, 0.14725755, 0.61708789, 0.11295755]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS with a Gaussian distribution requires two parameters: $\mu$ and $\sigma$ from the forecast distribution. Here, we just use the ensemble mean and ensemble spread.

In [37]:
crps_gaussian = xs.crps_gaussian(obs3, fct3.mean("member"), fct3.std("member"), dim=[])
print(crps_gaussian)

<xarray.DataArray 'var' (lat: 4, lon: 5)>
array([[0.09371309, 0.40393451, 0.05589101, 0.43197299, 0.65051953],
       [0.04923316, 0.19985396, 0.35666091, 0.09969302, 0.03126351],
       [0.21777917, 0.26356381, 0.54889593, 0.0152121 , 0.25893683],
       [0.12553405, 0.68803661, 0.15797719, 0.61047038, 0.0758741 ]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS quadrature metric requires a callable distribution function. Here we use `norm` from `scipy.stats`.

In [38]:
from scipy.stats import norm
crps_quadrature = xs.crps_quadrature(obs3, norm, dim=[])
print(crps_quadrature)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.3402993 , 0.57309997, 0.38930711, 0.23382702, 0.24561628],
       [0.40937975, 0.26107369, 0.24027277, 0.44370355, 0.31337576],
       [0.32028278, 0.46226592, 0.23962982, 0.25891839, 0.53080477],
       [0.24374443, 0.59624745, 0.60231   , 0.24599289, 0.32287915]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


We can also use a threshold Brier Score, to score hits over a certain threshold. Ranked Probability Score for two categories yields the same result.

In [46]:
threshold_brier_score = xs.threshold_brier_score(obs3, fct3, 0.5, dim=None)
print(threshold_brier_score)

<xarray.DataArray 'var' ()>
array(0.45)


In [40]:
brier_score = xs.brier_score(obs3>.5, (fct3>.5).mean('member'))
print(brier_score)

<xarray.DataArray 'var' ()>
array(0.45)


In [41]:
rps = xs.rps(obs3, fct3, bins=np.array([0.0, 0.5, 1.0]))
print(rps)

<xarray.DataArray 'histogram_category' ()>
array(0.45)


In [42]:
rank_histogram = xs.rank_histogram(obs3, fct3)
print(rank_histogram)

<xarray.DataArray 'histogram_rank' (rank: 4)>
array([8, 2, 5, 5])
Coordinates:
  * rank     (rank) float64 1.0 2.0 3.0 4.0


In [43]:
hist_event, hist_no_event = xs.discrimination(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(hist_event)

<xarray.DataArray (forecast_probability: 5)>
array([0.11111111, 0.55555556, 0.        , 0.22222222, 0.11111111])
Coordinates:
  * forecast_probability  (forecast_probability) float64 0.1 0.3 0.5 0.7 0.9


In [44]:
print(hist_no_event)

<xarray.DataArray (forecast_probability: 5)>
array([0.18181818, 0.18181818, 0.        , 0.27272727, 0.36363636])
Coordinates:
  * forecast_probability  (forecast_probability) float64 0.1 0.3 0.5 0.7 0.9


## Contingency-Based

To work with contingency-based scoring, first instantiate a `Contingency` object by passing in your observations, forecast, and observation/forecast bin edges. See https://www.cawcr.gov.au/projects/verification/#Contingency_table for more information.

In [21]:
dichotomous_category_edges = np.array([0, 0.5, 1]) # "dichotomous" mean two-category
dichotomous_contingency = xs.Contingency(obs, fct,
                                        dichotomous_category_edges,
                                        dichotomous_category_edges,
                                        dim=['lat','lon'])
dichotomous_contingency_table = dichotomous_contingency.table

In [22]:
dichotomous_contingency_table

<xarray.DataArray 'histogram_observations_forecasts' (time: 3, observations_category: 2, forecasts_category: 2)>
array([[[5, 6],
        [6, 3]],

       [[6, 5],
        [4, 5]],

       [[5, 5],
        [4, 6]]])
Coordinates:
  * time                          (time) object 2000-01-01 00:00:00 ... 2000-...
    observations_category_bounds  (observations_category) <U10 '[0.0, 0.5)' '...
    forecasts_category_bounds     (forecasts_category) <U10 '[0.0, 0.5)' '[0....
  * observations_category         (observations_category) int64 1 2
  * forecasts_category            (forecasts_category) int64 1 2

In [23]:
dichotomous_contingency_table.to_dataframe().pivot_table(index=['forecasts_category','forecasts_category_bounds'],
                                                         columns=['observations_category','observations_category_bounds']).round(2)

histogram_observations_forecasts  \
observations_category                                                       1   
observations_category_bounds                                       [0.0, 0.5)   
forecasts_category forecasts_category_bounds                                    
1                  [0.0, 0.5)                                            5.33   
2                  [0.5, 1.0)                                            5.33   

                                                         
observations_category                                 2  
observations_category_bounds                 [0.5, 1.0)  
forecasts_category forecasts_category_bounds             
1                  [0.0, 0.5)                      4.67  
2                  [0.5, 1.0)                      4.67

Scores based on the constructed contingency table can be called via class methods. The available methods are:

* `bias_score`
* `hit_rate`
* `false_alarm_ratio`
* `false_alarm_rate`
* `success_ratio`
* `threat_score`
* `equit_threat_score`
* `odds_ratio`
* `odds_ratio_skill_score`
* `accuracy`
* `heidke_score`
* `peirce_score`
* `gerrity_score`

Below, we share a few examples of these in action.

In [24]:
print(dichotomous_contingency.bias_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([1.        , 1.11111111, 1.1       ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [25]:
print(dichotomous_contingency.hit_rate())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.33333333, 0.55555556, 0.6       ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [26]:
print(dichotomous_contingency.false_alarm_rate())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.54545455, 0.45454545, 0.5       ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [27]:
print(dichotomous_contingency.odds_ratio_skill_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.41176471,  0.2       ,  0.2       ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


Now we can leverage multi-category edges to make use of some scores.

In [28]:
multi_category_edges = np.array([0, 0.25, 0.75, 1])
multicategory_contingency = xs.Contingency(obs, fct,
                                              multi_category_edges,
                                              multi_category_edges,
                                              dim=['lat','lon'])

In [29]:
print(multicategory_contingency.accuracy())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.25, 0.25, 0.5 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [30]:
print(multicategory_contingency.heidke_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.14503817, -0.25      ,  0.2481203 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [31]:
print(multicategory_contingency.peirce_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.1496063 , -0.24193548,  0.25      ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [32]:
print(multicategory_contingency.gerrity_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.15212912, -0.11160714,  0.25      ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Accessors

You can also use `xskillscore` as a method of your `xarray` Dataset.

In [33]:
ds = xr.Dataset()
ds["obs_var"] = obs
ds["fct_var"] = fct

In the case that your Dataset contains both your observation and forecast variable, just pass them as strings into the function.

In [34]:
print(ds.xs.pearson_r("obs_var", "fct_var", dim="time"))

<xarray.DataArray (lat: 4, lon: 5)>
array([[ 0.99509676, -0.88499394,  0.94083077,  0.96521259, -0.13696899],
       [-0.90613709,  0.51585291,  0.72875703,  0.19331043,  0.79754067],
       [-0.80112059, -0.95632624, -0.23640403, -0.57684283,  0.43389289],
       [ 0.00230351, -0.58970109, -0.87332763, -0.99992557, -0.31404248]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also pass in a separate Dataset that contains your observations or forecast variable.

In [35]:
ds = ds.drop_vars("fct_var")
print(ds.xs.pearson_r("obs_var", fct, dim="time"))

<xarray.DataArray (lat: 4, lon: 5)>
array([[ 0.99509676, -0.88499394,  0.94083077,  0.96521259, -0.13696899],
       [-0.90613709,  0.51585291,  0.72875703,  0.19331043,  0.79754067],
       [-0.80112059, -0.95632624, -0.23640403, -0.57684283,  0.43389289],
       [ 0.00230351, -0.58970109, -0.87332763, -0.99992557, -0.31404248]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4
